## Settings and run

In [ ]:
# Name/Path of the initial model.
MODEL_NAME = "stabilityai/stable-diffusion-2-base"

# Directory name to save model at.

OUTPUT_DIR = "musinsa/stable_diffusion_weights/modeloutput"


OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

# Start Training

In [ ]:
# You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

concepts_list = [
    {
        "instance_prompt":      "a person wearing  Musinsa_office_style clothes",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/office",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    },
    {
        "instance_prompt":      "a person wearing Musinsa_comfortable_style clothes",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/comfortable",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    }
]

concepts_list = [
    {
        "instance_prompt":      "a person wearing  Musinsa_tropical_style clothes",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/트로피컬",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    },
    {
        "instance_prompt":      "a person wearing Musinsa_christmas_style clothes",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/크리스마스",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    },
    {
        "instance_prompt":      "a person wearing clothes for Musinsa_picnic",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/피크닉",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    },
    {
        "instance_prompt":      "a person wearing clothes for Musinsa_party",
        "class_prompt":         "a person wearing clothes",
        "instance_data_dir":    "/content/drive/MyDrive/musinsa/key_pic/파티",
        "class_data_dir":       "/content/drive/MyDrive/musinsa/옷 코디"
    },
]


# `class_data_dir` contains regularization images
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("drive/MyDrive/musinsa/tropical_picnic_christmas_party.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

# Fine-tuning

In [ ]:
!accelerate launch /content/drive/MyDrive/musinsa/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=100 \
  --sample_batch_size=4 \
  --max_train_steps=3000 \
  --save_interval=10000 \
  --save_sample_prompt="a person wearing  Musinsa_tropical_style clothes for Musinsa_picnic" \
  --concepts_list="/content/drive/MyDrive/musinsa/tropical_picnic_christmas_party.json"

In [ ]:
from natsort import natsorted
from glob import glob
import os
WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
#Run to generate a grid of preview images from the last saved weights.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

weights_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(weights_folder) if f != "0"], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')
        
plt.tight_layout()
plt.savefig('grid.png', dpi=72)

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
# Run conversion.
ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
# Whether to convert to fp16, takes half the space (2GB).
fp16 = True 
if fp16:
    half_arg = "--half"

!python3 /content/drive/MyDrive/musinsa/convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

## Inference

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
model_path =   '/content/drive/MyDrive/musinsa/stable_diffusion_weights/modeloutput/3000'          # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")

g_cuda = None

In [ ]:
unet = UNet2DConditionModel.from_pretrained(
        model_path,
        subfolder="unet",
        revision="fp16",
        torch_dtype=torch.float32
    )

In [ ]:
# Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 715
g_cuda.manual_seed(seed)

In [ ]:
prompt = "a person wearing Musinsa_tropical_style clothes" 
negative_prompt = "blurry, bad fingers" 
num_samples = 4 
guidance_scale = 7.5
num_inference_steps = 50
height = 512 
width = 512 

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for i,img in enumerate(images):
    display(img)
    img.save(f'/content/drive/MyDrive/musinsa/create/{prompt}_{i}.jpg',"JPEG")

In [ ]:

import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)

In [ ]:
import shutil
from glob import glob
import os
for f in glob(OUTPUT_DIR+os.sep+"*"):
    if f != WEIGHTS_DIR:
        shutil.rmtree(f)
        print("Deleted", f)
for f in glob(WEIGHTS_DIR+"/*"):
    if not f.endswith(".ckpt") or not f.endswith(".json"):
        try:
            shutil.rmtree(f)
        except NotADirectoryError:
            continue
        print("Deleted", f)

In [ ]:
#@title Free runtime memory 
exit()